## Extracting Data

In [4]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [6]:
%%sql
SELECT *
FROM sqlite_master
WHERE type='table'

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


## Data Overview

In [7]:
%%sql
SELECT *
FROM facts
LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for the columns:

name — the name of the country.

area— the country's total area (both land and water).

area_land — the country's land area in square kilometers.

area_water — the country's water area in square kilometers.

population — the country's population.

population_growth— the country's population growth as a percentage.

birth_rate — the country's birth rate, or the number of births per year per 1,000 people.

death_rate — the country's death rate, or the number of death per year per 1,000 people.

## Calculating Summary Statistics

First, let's see the countries that has minimum and maximun population and population growrth rate.

In [10]:
%%sql
SELECT MIN(population) AS min_population, 
       MAX(population) AS max_population, 
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth
FROM facts;

 * sqlite:///factbook.db
Done.


min_population,max_population,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


Notice: 

1. a country with a population of zero person seems to be odd, decide to take a closer look
2. no country should have a population over 7.2 billion, that's almost all the population of th whole world

## Exploring Outliers

Use subqueries to zoom in on the countries with odd values:

In [23]:
%%sql
SELECT * 
FROM facts
WHERE population == (SELECT MIN(population) FROM facts)
OR population == (SELECT MAX(population) FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


Notice:

1. The table contains a row for Antarctica, which explains the population of 0.
2. It also seems like the table contains a row for the whole world, which explains the population of over 7.2 billion.


## Revised Summary Statistics

In [18]:
%%sql
SELECT MIN(population) AS min_population, 
       MAX(population) AS max_population, 
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth
FROM facts
WHERE name NOT IN ("Antarctica", "World");

 * sqlite:///factbook.db
Done.


min_population,max_population,min_pop_growth,max_pop_growth
48,1367485388,0.0,4.02


This time the summary statistics looks much more sence. From the max_population on table, we can guess the country should be China (13 billion).

## Analyzing Density

Next, we want to explore density. Density depends on the population and the country's area.

Let's look at the average values for these two columns:

In [27]:
%%sql
SELECT ROUND(AVG(population), 2) AS avg_population,
       ROUND(AVG(area), 2) AS avg_area
FROM facts
WHERE name NOT IN ("Antarctica", "World");

 * sqlite:///factbook.db
Done.


avg_population,avg_area
32377011.01,555093.55


Let's try to find countries that are densely populated. 

We identify countries that meets both of the follwing criteria:

1. The population is above average.
2. The area is below average.

In [29]:
%%sql
SELECT * 
FROM facts
WHERE population > (SELECT ROUND(AVG(population), 2) FROM facts WHERE name NOT IN ("Antarctica", "World"))
AND area < (SELECT ROUND(AVG(area), 2) FROM facts WHERE name NOT IN ("Antarctica", "World"));

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


These are generally known to be densely populated, so we have confidence in our results!

## Analyzing Population Growth

Lastly, let's take a look at the popualtion growth, which often correlates to the productivity and demographic dividend of a country.


We would like to know:
1. Which countries have a higher death rate than birth rate?
2. Which countries will add the most people to their populations next year?

In [32]:
%%sql
SELECT * 
FROM facts
WHERE death_rate > birth_rate
ORDER BY death_rate - birth_rate DESC;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
26,bu,Bulgaria,110879,108489,2390,7186893,0.58,8.92,14.44,0.29
153,ri,Serbia,77474,77474,0,7176794,0.46,9.08,13.66,0.0
96,lg,Latvia,64589,62249,2340,1986705,1.06,10.0,14.31,6.26
102,lh,Lithuania,65300,62680,2620,2884433,1.04,10.1,14.27,6.27
183,up,Ukraine,603550,579330,24220,44429471,0.6,10.72,14.46,2.25
75,hu,Hungary,93028,89608,3420,9897541,0.22,9.16,12.73,1.33
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
158,si,Slovenia,20273,20151,122,1983412,0.26,8.42,11.37,0.37
142,ro,Romania,238391,229891,8500,21666350,0.3,9.14,11.9,0.24
44,hr,Croatia,56594,55974,620,4464844,0.13,9.45,12.18,1.39


It seems like countries in Europe tend to have a higher death_rate than birth_rate.

In [37]:
%%sql
SELECT name, 
       ROUND(population * population_growth, 2) AS pop_next_year
FROM facts
WHERE name NOT IN ("Antarctica", "World")
ORDER BY pop_next_year DESC
LIMIT 5;

 * sqlite:///factbook.db
Done.


name,pop_next_year
India,1527068612.48
China,615368424.6
Nigeria,444827037.2
Pakistan,290665336.62
Ethiopia,287456216.91


Now we have the top 5 countries that will add most people next year.